# Exploración de Datos de Seguridad en México

Este notebook sirve para probar la descarga de datos de seguridad y realizar una exploración inicial antes de procesar los datos.

## Fuentes de datos:
1. **Percepción de Inseguridad** - INEGI/ENVIPE (2011-2025)
2. **Incidencia Delictiva Estatal** - SESNSP (2015-2025)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

: 

## 1. Configuración de Rutas

In [ ]:
# Rutas del proyecto
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
DATA_INTERIM = PROJECT_ROOT / 'data' / 'interim'

print(f"Proyecto: {PROJECT_ROOT}")
print(f"Datos raw: {DATA_RAW}")
print(f"Datos procesados: {DATA_PROCESSED}")

## 2. Prueba de Descarga de Datos

Primero probamos la descarga usando el script (opcional si ya tienes los datos)

In [ ]:
# Ejecutar descarga (comentar si ya tienes los datos)
# !python datos_seguridad_mexico.py --token 32805429-135c-9311-70c1-0b963c6f8317

## 3. Cargar Datos Raw

### 3.1 Percepción de Inseguridad

In [ ]:
# Cargar datos de percepción
df_percepcion = pd.read_csv(DATA_RAW / 'indicador_inseguridad_estados.csv')

print(f"Shape: {df_percepcion.shape}")
print(f"\nColumnas: {df_percepcion.columns.tolist()}")
print(f"\nPrimeras filas:")
df_percepcion.head(10)

In [ ]:
# Información general
df_percepcion.info()

In [ ]:
# Estadísticas descriptivas
df_percepcion.describe()

In [ ]:
# Verificar valores nulos
print("Valores nulos por columna:")
print(df_percepcion.isnull().sum())
print(f"\nPorcentaje de nulos:")
print(df_percepcion.isnull().sum() / len(df_percepcion) * 100)

In [ ]:
# Rango de años
print(f"Años disponibles: {df_percepcion['año'].min()} - {df_percepcion['año'].max()}")
print(f"\nEntidades únicas: {df_percepcion['entidad'].nunique()}")
print(f"\nListado de entidades:")
print(sorted(df_percepcion['entidad'].unique()))

### 3.2 Incidencia Delictiva

In [ ]:
# Cargar datos de incidencia delictiva
df_delictiva = pd.read_csv(DATA_RAW / 'incidencia_delictiva_estatal_2015_2025.csv')

print(f"Shape: {df_delictiva.shape}")
print(f"\nColumnas: {df_delictiva.columns.tolist()}")
print(f"\nPrimeras filas:")
df_delictiva.head(10)

In [ ]:
# Información general
df_delictiva.info()

In [ ]:
# Valores nulos
print("Valores nulos por columna:")
print(df_delictiva.isnull().sum())

In [ ]:
# Explorar estructura (depende de las columnas reales)
# Mostrar las primeras columnas para entender la estructura
print("Columnas del dataset:")
for col in df_delictiva.columns:
    print(f"  - {col}")

## 4. Visualización Exploratoria

### 4.1 Percepción de Inseguridad

In [ ]:
# Evolución temporal a nivel nacional
df_nacional = df_percepcion[df_percepcion['entidad'] == 'Nacional']

plt.figure(figsize=(12, 6))
plt.plot(df_nacional['año'], df_nacional['valor'], marker='o', linewidth=2, markersize=8)
plt.title('Evolución de la Percepción de Inseguridad a Nivel Nacional', fontsize=14, fontweight='bold')
plt.xlabel('Año', fontsize=12)
plt.ylabel('Personas que perciben inseguridad (por cada 100,000)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Top 10 estados con mayor percepción (año más reciente)
año_reciente = df_percepcion['año'].max()
df_reciente = df_percepcion[
    (df_percepcion['año'] == año_reciente) & 
    (df_percepcion['entidad'] != 'Nacional')
].sort_values('valor', ascending=False).head(10)

plt.figure(figsize=(12, 6))
plt.barh(df_reciente['entidad'], df_reciente['valor'])
plt.title(f'Top 10 Estados con Mayor Percepción de Inseguridad ({año_reciente})', 
          fontsize=14, fontweight='bold')
plt.xlabel('Personas que perciben inseguridad (por cada 100,000)', fontsize=12)
plt.ylabel('Estado', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Evolución de algunos estados seleccionados
estados_interes = ['Ciudad de México', 'Estado de México', 'Jalisco', 'Nuevo León', 'Guanajuato']
df_estados = df_percepcion[df_percepcion['entidad'].isin(estados_interes)]

plt.figure(figsize=(14, 7))
for estado in estados_interes:
    df_estado = df_estados[df_estados['entidad'] == estado]
    plt.plot(df_estado['año'], df_estado['valor'], marker='o', label=estado, linewidth=2)

plt.title('Evolución de la Percepción de Inseguridad por Estado', fontsize=14, fontweight='bold')
plt.xlabel('Año', fontsize=12)
plt.ylabel('Personas que perciben inseguridad (por cada 100,000)', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 4.2 Análisis de Calidad de Datos

In [ ]:
# Verificar completitud de datos por entidad
años_esperados = set(range(df_percepcion['año'].min(), df_percepcion['año'].max() + 1))
print(f"Años esperados: {len(años_esperados)} años")
print(f"Rango: {min(años_esperados)} - {max(años_esperados)}")

print("\nCompletitud de datos por entidad:")
for entidad in sorted(df_percepcion['entidad'].unique()):
    df_ent = df_percepcion[df_percepcion['entidad'] == entidad]
    años_disponibles = set(df_ent['año'].unique())
    años_faltantes = años_esperados - años_disponibles
    pct_completo = len(años_disponibles) / len(años_esperados) * 100
    
    status = "✓" if pct_completo == 100 else "⚠"
    print(f"{status} {entidad:25s}: {pct_completo:5.1f}% ({len(años_disponibles)}/{len(años_esperados)} años)")
    
    if años_faltantes:
        print(f"  Años faltantes: {sorted(años_faltantes)}")

In [ ]:
# Detectar valores atípicos (outliers)
Q1 = df_percepcion['valor'].quantile(0.25)
Q3 = df_percepcion['valor'].quantile(0.75)
IQR = Q3 - Q1

outliers_lower = Q1 - 1.5 * IQR
outliers_upper = Q3 + 1.5 * IQR

df_outliers = df_percepcion[
    (df_percepcion['valor'] < outliers_lower) | 
    (df_percepcion['valor'] > outliers_upper)
]

print(f"Valores atípicos detectados: {len(df_outliers)}")
if len(df_outliers) > 0:
    print("\nRegistros con valores atípicos:")
    print(df_outliers[['año', 'entidad', 'valor']].sort_values('valor', ascending=False))

In [ ]:
# Boxplot por año
plt.figure(figsize=(14, 6))
df_percepcion[df_percepcion['entidad'] != 'Nacional'].boxplot(
    column='valor', by='año', figsize=(14, 6)
)
plt.suptitle('')
plt.title('Distribución de Percepción de Inseguridad por Año', fontsize=14, fontweight='bold')
plt.xlabel('Año', fontsize=12)
plt.ylabel('Valor del indicador', fontsize=12)
plt.tight_layout()
plt.show()

## 5. Conclusiones de la Exploración

### Observaciones clave:
1. **Completitud de datos**: [Analizar resultados]
2. **Valores nulos**: [Analizar resultados]
3. **Tendencias temporales**: [Analizar resultados]
4. **Outliers**: [Analizar resultados]

### Próximos pasos:
1. Crear script de procesamiento para limpiar y transformar datos
2. Implementar validaciones de calidad
3. Generar datasets tidy para análisis